# RAG Evaluation

## Testset preparing

In [1]:
from operations.storages import ChromaDBOperations
import pandas as pd

chroma_oper = ChromaDBOperations()
documents = chroma_oper.get_all_documents()


all_docs = pd.DataFrame([doc.get("document") for doc in documents], columns=["text"])

/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedder = HuggingFaceEmbeddings(model_name=embedder_dir)
/home/wiktor/projects/rag-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(all_docs)

75

In [5]:
import giskard
from giskard.llm.client.openai import OpenAIClient

# ai_client = OpenAIClient(model="gpt-4o-mini", client=evaluation_llm)

# giskard.llm.set_default_client(ai_client)


In [6]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(data=all_docs) #, llm_client=ai_client)

2025-11-11 14:58:46,274 pid:17360 MainThread giskard.llm.embeddings INFO     No embedding model set though giskard.llm.set_embedding_model. Defaulting to openai/text-embedding-3-small since OPENAI_API_KEY is set.


In [8]:
from giskard.rag import generate_testset

testset = generate_testset(knowledge_base, 
                           num_questions=50)

Generating questions: 100%|██████████| 50/50 [03:01<00:00,  3.63s/it]


In [10]:
testset.save("evaluation/pre-processed/test-set.jsonl")

## Processing testset

In [1]:
from giskard.rag import QATestset

testset = QATestset.load("evaluation/pre-processed/test-set.jsonl")

/home/wiktor/projects/rag-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_df = testset.to_pandas()
test_df = test_df.drop(['conversation_history'], axis=1)
test_df['reference_context'] = test_df['reference_context'].str.replace(r'^Document \d+: ', '', regex=True)
test_df['metadata'] = test_df['metadata'].apply(lambda x: x.get('question_type'))
test_df['retrieved_context'] = [[] for _ in range(len(test_df))]

test_df.head()

,question,reference_answer,reference_context,metadata,retrieved_context
id,,,,,
5702a02d-c421-4d92-842b-626514612f5c,What happens if a player lands on a property t...,"If a player does not wish to buy the property,...",You receive the Title Deed card showing owners...,simple,[]
64e45cd9-bd68-4627-bc7e-c3ba341a5939,What was Lizzie's creation that she sought to ...,Lizzie sought to patent the Landlord's Game.,Her vision was an embrace of dualism and conta...,simple,[]
111d9dc2-8569-44c7-9c27-73ba7f7168fe,What is the main contribution of the study men...,The study introduced an adaptive retrieval-aug...,"4 Conclusion. In this study, we introduced an ...",simple,[]
2df4fd1b-a71a-4535-9274-96a9e650ee7f,"What is the purpose of the dynamic, fine-grain...","The dynamic, fine-grained reward function is d...",The bandit framework requires only partial inf...,simple,[]
42e9f5ac-3b5e-441d-b661-67b455122da5,What happens if your token lands on the 'Go to...,When your token lands on the space marked 'Go ...,. . ( I ) your token lands on the space marked...,simple,[]


In [3]:
from operations.storages import ChromaDBOperations


chroma_oper = ChromaDBOperations()
retriever = chroma_oper.get_retriever(k=4)

def get_retrieved_context(query):
    docs = retriever.invoke(query)
    return [doc.page_content for doc in docs]

test_df['retrieved_context'] = test_df['question'].apply(get_retrieved_context)

/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedder = HuggingFaceEmbeddings(model_name=embedder_dir)
/home/wiktor/projects/rag-bot/src/operations/storages/chromadb_operations.py:46: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(


In [4]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from agent.graphs.chat_agent import agent

async def generated_answer(query: str):
    response = await agent.ainvoke(input={"query": query})
    return response.get('messages')[-1].content

async def fill_generated_answers(df):
    results = []
    for query in df["question"]:
        res = await generated_answer(query)
        results.append(res)
    df["generated_answer"] = results
    return df

test_df = asyncio.run(fill_generated_answers(test_df))

In [ ]:
# test_df['reference_context'] = test_df['reference_context'].apply(lambda x: [x])

In [6]:
df_ragas = test_df.rename(columns={
    "generated_answer": "response",
    "reference_answer": "ground_truths",
    "retrieved_context": "retrieved_contexts",
    "reference_context": "reference"
})

In [7]:
df_ragas

,question,ground_truths,reference,metadata,retrieved_contexts,response
id,,,,,,
5702a02d-c421-4d92-842b-626514612f5c,What happens if a player lands on a property t...,"If a player does not wish to buy the property,...",You receive the Title Deed card showing owners...,simple,[If the player who draws it does not wish to u...,If a player lands on a property they do not wi...
64e45cd9-bd68-4627-bc7e-c3ba341a5939,What was Lizzie's creation that she sought to ...,Lizzie sought to patent the Landlord's Game.,Her vision was an embrace of dualism and conta...,simple,[Her vision was an embrace of dualism and cont...,Lizzie sought to patent her creation known as ...
111d9dc2-8569-44c7-9c27-73ba7f7168fe,What is the main contribution of the study men...,The study introduced an adaptive retrieval-aug...,"4 Conclusion. In this study, we introduced an ...",simple,"[When access to information is made equal, gre...",The main contribution of the study is the deve...
2df4fd1b-a71a-4535-9274-96a9e650ee7f,"What is the purpose of the dynamic, fine-grain...","The dynamic, fine-grained reward function is d...",The bandit framework requires only partial inf...,simple,[The bandit framework requires only partial in...,"The purpose of the dynamic, fine-grained rewar..."
42e9f5ac-3b5e-441d-b661-67b455122da5,What happens if your token lands on the 'Go to...,When your token lands on the space marked 'Go ...,. . ( I ) your token lands on the space marked...,simple,[. . ( I ) your token lands on the space marke...,"If your token lands on the ""Go to Jail"" space ..."
5d1b0f53-9f31-4ac7-8742-295b81988bc5,What strategy is used to balance exploration a...,An epsilon-greedy strategy is used to balance ...,2.2 Query Encoding and Arm Selection. To overc...,simple,[The bandit framework requires only partial in...,The strategy used to balance exploration and e...
67ee11be-f320-4314-97bc-3f1a710cb023,What are the two stages involved in the RAG pr...,The RAG process includes a retrieval stage whe...,2.1 Preliminaries. The RAG process (Lewis et a...,simple,[2.1 Preliminaries. The RAG process (Lewis et ...,The RAG process consists of two main stages:\n...
319eaf0b-00db-402b-9b59-f7bd0d819848,What features did The Landlord's Game include?,"The Landlord's Game included play money, deeds...",The Landlord's Game. Photograph: Tom Forsyth. ...,simple,[The Landlord's Game. Photograph: Tom Forsyth....,The Landlord's Game included several notable f...
a92b240e-942c-40a3-9d56-99395b894763,What does the Guardian suggest about the impac...,"When access to information is made equal, grea...","When access to information is made equal, grea...",simple,"[When access to information is made equal, gre...",The Guardian suggests that equal access to inf...


In [9]:
import json

post_testset = df_ragas.to_dict(orient='records')
with open('evaluation/post-processed/processed-testset.json', 'w') as f:
    json.dump(post_testset, f, indent=2)

## Ragas eval

In [10]:
import pandas as pd
df = pd.read_json('evaluation/post-processed/processed-testset.json', orient='records')
df.head()

,question,ground_truths,reference,metadata,retrieved_contexts,response
0,What happens if a player lands on a property t...,"If a player does not wish to buy the property,...",You receive the Title Deed card showing owners...,simple,[If the player who draws it does not wish to u...,If a player lands on a property they do not wi...
1,What was Lizzie's creation that she sought to ...,Lizzie sought to patent the Landlord's Game.,Her vision was an embrace of dualism and conta...,simple,[Her vision was an embrace of dualism and cont...,Lizzie sought to patent her creation known as ...
2,What is the main contribution of the study men...,The study introduced an adaptive retrieval-aug...,"4 Conclusion. In this study, we introduced an ...",simple,"[When access to information is made equal, gre...",The main contribution of the study is the deve...
3,"What is the purpose of the dynamic, fine-grain...","The dynamic, fine-grained reward function is d...",The bandit framework requires only partial inf...,simple,[The bandit framework requires only partial in...,"The purpose of the dynamic, fine-grained rewar..."
4,What happens if your token lands on the 'Go to...,When your token lands on the space marked 'Go ...,. . ( I ) your token lands on the space marked...,simple,[. . ( I ) your token lands on the space marke...,"If your token lands on the ""Go to Jail"" space ..."


In [11]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
)
from ragas import evaluate
from datasets import Dataset

dataset = Dataset.from_pandas(df[[
    "question",
    "ground_truths",
    "reference",
    "retrieved_contexts",
    "response"
]])

results = evaluate(
    dataset, 
    metrics=[faithfulness, context_precision, context_recall, answer_relevancy, answer_correctness],
    show_progress=True
    )

Evaluating:   0%|          | 0/250 [00:00<?, ?it/s]

2025-11-11 15:12:55,768 pid:33831 MainThread ragas.prompt.pydantic_prompt WARNING  LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Evaluating:   0%|          | 1/250 [00:05<21:34,  5.20s/it]

2025-11-11 15:12:58,685 pid:33831 MainThread ragas.prompt.pydantic_prompt WARNING  LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
2025-11-11 15:13:01,211 pid:33831 MainThread ragas.prompt.pydantic_prompt WARNING  LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Evaluating:   5%|▌         | 13/250 [00:47<12:50,  3.25s/it]

...

Evaluating:  94%|█████████▎| 234/250 [10:18<00:24,  1.56s/it]

2025-11-11 15:23:12,835 pid:33831 MainThread ragas.prompt.pydantic_prompt WARNING  LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Evaluating:  95%|█████████▍| 237/250 [10:29<00:36,  2.84s/it]

2025-11-11 15:23:25,780 pid:33831 MainThread ragas.prompt.pydantic_prompt WARNING  LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


Evaluating: 100%|██████████| 250/250 [11:25<00:00,  2.74s/it]


In [ ]:
df = results.to_pandas()

df['context_f1'] = 2 * (df['context_precision'] * df['context_recall']) / (
    df['context_precision'] + df['context_recall']
)

df['context_f1'] = df['context_f1'].fillna(0.0)

df.head()

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness,context_f1
0,What happens if a player lands on a property t...,[If the player who draws it does not wish to u...,If a player lands on a property they do not wi...,You receive the Title Deed card showing owners...,1.000000,1.000000,1.000000,0.966812,0.213965,1.000000
1,What was Lizzie's creation that she sought to ...,[Her vision was an embrace of dualism and cont...,Lizzie sought to patent her creation known as ...,Her vision was an embrace of dualism and conta...,0.600000,1.000000,0.333333,0.808651,0.355482,0.500000
2,What is the main contribution of the study men...,"[When access to information is made equal, gre...",The main contribution of the study is the deve...,"4 Conclusion. In this study, we introduced an ...",0.285714,0.000000,0.500000,0.888146,0.777891,0.000000
3,"What is the purpose of the dynamic, fine-grain...",[The bandit framework requires only partial in...,"The purpose of the dynamic, fine-grained rewar...",The bandit framework requires only partial inf...,0.583333,0.916667,1.000000,1.000000,0.891704,0.956522
4,What happens if your token lands on the 'Go to...,[. . ( I ) your token lands on the space marke...,"If your token lands on the ""Go to Jail"" space ...",. . ( I ) your token lands on the space marked...,1.000000,1.000000,0.800000,0.996573,0.771582,0.888889


In [14]:
metrics_dict = {
    "faithfulness": round(df['faithfulness'].mean(), 4),
    "context_precision": round(df['context_precision'].mean(), 4),
    "context_recall": round(df['context_recall'].mean(), 4),
    "context_f1": round(df['context_f1'].mean(), 4),
    "answer_relevancy": round(df['answer_relevancy'].mean(), 4),
    "answer_correctness": round(df['answer_correctness'].mean(), 4)
}
print(metrics_dict)

{'faithfulness': 0.8536, 'context_precision': 0.8156, 'context_recall': 0.7264, 'context_f1': 0.7349, 'answer_relevancy': 0.8122, 'answer_correctness': 0.5735}


In [13]:
import json

metrics = df.to_dict(orient='records')
with open('evaluation/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)